In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root_scalar

gamma = 50
alpha = 1e6

# defining the Lennard jones potential
def v(x):
    return 4*(1/x**12 - 1/x**6)

# internal part of the Schrodinger eqn, in the Numerov general form y"(t) = g(t)y
def g(x, ek):
    return gamma**2 * abs(v(x) - ek)






# ACTUAL CALCULATIONS START HERE
wkb_Energies = [
    -0.94744497,  # Energy at level 0
    -0.84765555,  # Energy at level 1
    -0.75479743,  # Energy at level 2
    -0.66867222,  # Energy at level 3
    -0.58907396,  # Energy at level 4
    -0.51579425,  # Energy at level 5
    -0.44861756,  # Energy at level 6
    -0.3873236,   # Energy at level 7
    -0.33168588,  # Energy at level 8
    -0.28147264   # Energy at level 9
]


# defining the x-grid
dx = 0.01 
x = np.arange(0.99, 3, dx)
vx = [v(i) for i in x ]

plt.plot(x, vx,color='k', label="LJ Potential", linewidth = 1)
plt.xlabel("x")
plt.ylabel("V(x)")


for ek in wkb_Energies:

    """ Calculating the CTPs, root_scalar returns object with execulation results, unpacking needed for actual root """ 
    ctp_left = root_scalar(lambda x: v(x)-ek, method='secant', x0=1.0, x1=1.0 + 1e-4)
    ctp_right = root_scalar(lambda x: v(x)-ek, method='secant', x0=1.25, x1=1.25 + 1e-4)

    xa = ctp_left.root
    xb = ctp_right.root

    ## alternative mechanism for finding the CTPs : simplifies computations
    # for i in range(1, len(x)):
    #     if (vx[i-1]>ek and ek>vx[i]):
    #         ctpA = x[i]
    #     if (vx[i-1]<ek and ek<vx[i]):
    #         ctpB = x[i]      
    # ctpA, ctpB

    """Calculation of the integration limits for the wavefunction ODE"""
    # applying bisection method to calculate the start limit, xs
    # func: sqrt{g(xs)} (xa-xs) ln A
    xs = root_scalar(
        lambda x: np.sqrt(g(x,ek)) * (xa - x) - np.log(alpha),
        bracket=[0.1,3],
        method='bisect',
    )
    # applying the same for end limit
    # func: sqrt{g(xe)} (xe-xb) ln A
    xe = root_scalar(
        lambda x: np.sqrt(g(x,ek)) * (x - xb) - np.log(alpha),
        bracket=[0.1,3],
        method='bisect',
    )
    xs = xs.root #unpacking the root_scalar object
    xe = xe.root
    
    plt.scatter(xa, ek, marker="o", c='r', s=25)
    plt.scatter(xb, ek, marker="o",c='r', s=25 )
    plt.scatter(xs, ek, marker='s', color = 'b', s=30)
    plt.scatter(xe, ek, marker='s', color= 'b', s=30)
    plt.hlines(y=ek, xmin=xs, xmax=xe, color='r', linestyle='--', linewidth=0.5, label=f"{ek}")

plt.grid()
plt.legend()


SyntaxError: invalid syntax (1237204118.py, line 21)

In [ ]:
def return_limits_for_E(ek):
    xs = root_scalar(
            lambda x: np.sqrt(g(x,ek)) * (xa - x) - np.log(alpha),
            bracket=[0.1,3],
            method='bisect',
        )
        # applying the same for end limit
        # func: sqrt{g(xe)} (xe-xb) ln A
    xe = root_scalar(
            lambda x: np.sqrt(g(x, ek)) * (x - xb) - np.log(alpha),
            bracket=[0.1,3],
            method='bisect',
        )
    xs = xs.root #unpacking the root_scalar object
    xe = xe.root
    return xs,xe

In [ ]:
# now we can proceed towards the integration process.
# Re-implement the refined explicit Numerov propagator
# re configured the numerov func for SE 
def numerov_for_SE(func,phi0, phi1, h, t):
    """
    Refined explicit Numerov propagator for solving y''(t) = g(t)y(t).
    
    Parameters:
    - func: The function g(t), such that y''(t) = g(t)y(t).
    - y0: Initial condition y(t0).
    - v0: Initial velocity y'(t0).
    - h: Step size for the numerical method.
    - t: Array of time points.
    
    Returns:
    - y: Array of dependent variable values computed using the explicit Numerov method.
    """
    y = np.zeros_like(t)
    
    y[0] = phi0 # two start points, as indicated by the setup
    y[1] = phi1  

    for n in range(1, len(t) - 1):
        # Compute coefficients a, b, c
        g_n = func(t[n])
        g_n_minus_1 = func(t[n - 1])
        g_n_plus_1 = func(t[n + 1])

        a = 2 + (5 / 6) * g_n * h**2
        b = 1 - (1 / 12) * g_n_minus_1 * h**2
        c = 1 - (1 / 12) * g_n_plus_1 * h**2

        # Update y[n+1] explicitly
        y[n + 1] = (a * y[n] - b * y[n - 1]) / c

    return y

def g_for_SE(x,ek):
    return gamma**2 * (v(x) - ek)
    
    
# plotting all of this for a given energy
iter = 1
k = 6
e_upper = 0
e_lower = -1.0

phi_x_e_prev = 0 

while iter <= 100:
    e_mid = (e_upper + e_lower)/2
    
    
    # making calculations for current value of e_mid
    xs, xe = return_limits_for_E(e_mid)
    dx = 0.001
    x_range = np.arange(xs-dx, xe+dx, dx)
    phi = numerov_for_SE(
        lambda x: g_for_SE(x,e_mid),
        phi0 = 0,
        phi1 = 1,
        h=dx,
        t=x_range
    )
    # phi = phi / np.dot(phi,phi)
    
    # plt.plot(x_range[0: x_range.size-10], phi[0: x_range.size-10], label = f"e = {e_mid}")
    
    nodes = 0
    for i in range(0, phi.size-1):
        if(phi[i]*phi[i+1] < 0):
            nodes = nodes + 1 
    
    
    # print(f"nodes: {nodes} \t phi[x_e]: {phi[-1]}")
    
    if nodes > k:
        e_upper = e_mid
    else:
        e_lower = e_mid
    
    if k == nodes and abs(phi[-1])<1e-3:
        print("TERMINATED, iter:", iter)
        break
    
    if nodes == k and abs(phi_x_e_prev - phi[-1]) < 1e-12:
        print("CONVERGED, iter :", iter)
        break
    
    phi_x_e_prev = phi[-1]
    iter = iter + 1

# normalisation of the wavefunction
phi = phi / np.sqrt(np.dot(phi,phi))
print("Energy:", ek)
print("NORM OF THE FINAL WAVEFunc:",np.dot(phi, phi)) #verification of norm, should be

# plotting the lennard jones potential
plt.plot(x, vx,color='k', label="LJ Potential", linewidth = 1)
plt.xlabel("x")
plt.ylabel("V(x)")

xs, xe = return_limits_for_E(ek=e_mid) # plotting the wavefunction for the final value of e
plt.scatter(xa, ek, marker="o", c='r', s=25)
plt.scatter(xb, ek, marker="o",c='r', s=25 )
plt.scatter(xs, ek, marker='s', color = 'b', s=30)
plt.scatter(xe, ek, marker='s', color= 'b', s=30)

plt.hlines(y=ek, xmin=xs, xmax=xe, color='r', linestyle='--', linewidth=0.5, label=f"e={ek}, nodes/k = {nodes}/{k}")
phi_shifted = phi - abs(ek)
plt.plot(x_range, phi_shifted)

plt.grid()
plt.legend()

    

NameError: name 'xa' is not defined